In [1]:
from scipy.io import wavfile
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import random
import time
from glob import glob

In [2]:
wave_point = 256

y_size = int(wave_point * 3)
x_size = int(y_size / 4)
zip_y_size = int(y_size / 2)

In [3]:
def data_gap_format(data):
    hight_pitch, low_pitch, gap = 0, 0, 0

    for i in data:
        if i > hight_pitch and i > 0:
            hight_pitch = i
        if i < low_pitch and i < 0:
            low_pitch = i

    gap = hight_pitch if hight_pitch > (low_pitch * -1) else (low_pitch * -1)

    return hight_pitch, low_pitch, gap, [i / gap for i in data]

def data_gap_resize(data, gap):
    return [i * gap for i in data]

In [4]:
def int_to_matrix(value_int, gap):
    if value_int == 0:
        return [0, 128 / gap, 0]
    if value_int > 0:
        return [value_int, 0, 0]
    if value_int < 0:
        return [0, 0, value_int * -1]
    
    return 0
    # return [0, 128 / gap, 0] if value_int == 0 else [value_int, 0, 0] if value_int > 0 else [0, 0, value_int * -1]

def matrix_to_int(matrix_int, gap):
    value = np.amax(matrix_int)
    index = list(matrix_int).index(value)
    
    if index == 0:
        return value * gap
    if index == 1:
        return 0
    if index == 2:
        return value * gap * -1
    
    return 0

In [5]:
def pick_x_data(dataset):
    return [dataset[i] for i in range(0, len(dataset), 4)]

def pick_wave_data(dataset, index):
    return dataset[index - wave_point: index]

In [6]:
# def view_predict_input(dataset):
#     return np.array([[i, 0, 0, 0] for i in dataset]).reshape(len(dataset * 4))

In [7]:
# wave data load

wave_files = glob("kevin_wave/*.wav")

x_wave = []

for index, file in enumerate(wave_files):
    _, contents = wavfile.read(file)

    x_wave = x_wave + [i[0] for i in contents]
    
hight_pitch, low_pitch, gap, x_wave = data_gap_format(x_wave)

x_wave = np.array(x_wave)
np.save("gan_64_x_tmp_save.npy", x_wave)

In [8]:
# # load default data

# hight_pitch, low_pitch, gap = 24811, -26038, 26038

# x_wave = np.load("gan_64_x_tmp_save.npy")
# x_wave = x_wave.tolist()

In [9]:
# tensorflow

import tensorflow as tf
import tflearn

In [10]:
# Generator
def generator(x, reuse=False):
    with tf.variable_scope('Generator', reuse=reuse):
        x = tflearn.fully_connected(x, zip_y_size, activation='relu')
        x = tflearn.fully_connected(x, y_size, activation='sigmoid')
        return x


# Discriminator
def discriminator(x, reuse=False):
    with tf.variable_scope('Discriminator', reuse=reuse):
        x = tflearn.fully_connected(x, zip_y_size, activation='relu')
        x = tflearn.fully_connected(x, 1, activation='sigmoid')
        return x

In [11]:
gen_input = tflearn.input_data(shape=[None, x_size], name='input_noise')
disc_input = tflearn.input_data(shape=[None, y_size], name='disc_input')

gen_sample = generator(gen_input)
disc_real = discriminator(disc_input)
disc_fake = discriminator(gen_sample, reuse=True)

# Define Loss
disc_loss = -tf.reduce_mean(tf.log(disc_real) + tf.log(1. - disc_fake))
gen_loss = -tf.reduce_mean(tf.log(disc_fake))

gen_vars = tflearn.get_layer_variables_by_scope('Generator')
gen_model = tflearn.regression(gen_sample, placeholder=None, optimizer='adam',
                               loss=gen_loss, trainable_vars=gen_vars,
                               batch_size=64, name='target_gen', op_name='GEN')
disc_vars = tflearn.get_layer_variables_by_scope('Discriminator')
disc_model = tflearn.regression(disc_real, placeholder=None, optimizer='adam',
                                loss=disc_loss, trainable_vars=disc_vars,
                                batch_size=64, name='target_disc', op_name='DISC')

gan = tflearn.DNN(gen_model)

In [12]:
X = []
Y = []
auto_save_time = 10
last_time_save = time.time()

for i in range(len(x_wave)):
    this_data = pick_wave_data(x_wave, i)
    
    if (np.all(np.array(this_data)==0)):
        continue
    
    tmp_y = np.array([int_to_matrix(i, gap) for i in this_data]).reshape(y_size)
    
    tmp_x = pick_x_data(np.array(this_data, copy=True))
    tmp_x = np.array([int_to_matrix(i, gap) for i in tmp_x]).reshape(x_size)
    
    Y.append(tmp_y)
    X.append(tmp_x)
    
    if len(X) > 1000:
        gan.fit(X_inputs={gen_input: X, disc_input: Y}, Y_targets=None, n_epoch=100)
        X = []
        Y = []
        
    if (last_time_save + (60 * auto_save_time)) < time.time():
        last_time_save = time.time()
        gan.save("tmp_gan_64_save")
        
        print("auto save")
        
        # test code
        break

Training Step: 97599  | total loss: nan | time: 0.067s
| GEN | epoch: 6100 | loss: nan -- iter: 0960/1001
| DISC | epoch: 6100 | loss: nan -- iter: 0960/1001
Training Step: 97600  | total loss: nan | time: 0.072s
| GEN | epoch: 6100 | loss: nan -- iter: 1001/1001
| DISC | epoch: 6100 | loss: nan -- iter: 1001/1001
--
INFO:tensorflow:/home/lewis/wave_test/tmp_gan_64_save is not in all_model_checkpoint_paths. Manually adding it.
auto save


In [13]:
gan.load("tmp_gan_64_save")

test_real_data = pick_wave_data(x_wave, 88888)

tmp = np.array([int_to_matrix(i, gap) for i in test_real_data]).reshape(y_size)

tmp_x = pick_x_data(np.array(tmp, copy=True))

gan_predict = gan.predict([[tmp_x]])

INFO:tensorflow:Restoring parameters from /home/lewis/wave_test/tmp_gan_64_save


In [14]:
# [matrix_to_int(gan_predict[0][i:i+3], gap) for i in range(0, len(gan_predict[0]), 3)]

In [15]:
# plt.figure(1)
# plt.title('wave_predict')
# plt.plot(np.array([[matrix_to_int(tmp_x[i:i+3], gap), 0, 0, 0] for i in range(0, len(tmp_x), 3)]).reshape(y_size))
# plt.show()

plt.figure(1)
plt.title('wave_predict')
plt.plot([matrix_to_int(gan_predict[0][i:i+3], gap) for i in range(0, len(gan_predict[0]), 3)])
plt.show()

plt.figure(1)
plt.title('wave_real')
plt.plot(test_real_data)
plt.show()

ValueError: nan is not in list

192